In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
import time
from matplotlib import pyplot
%run lstm.ipynb

## Гиперпараметры

In [ ]:
decimation = 30

num_epochs = 150
num_repeat = 1

years = 1.5 # <- окно обучения 7.5 лет
output_dim = int(years * 365 / decimation)
input_dim = int(output_dim)
hidden_dim = int(input_dim * 1.5)

learn_rate = 0.5
optimizer = 'adagrad'
normalize = True

print(input_dim, hidden_dim, output_dim)

## Чтение данных
Пример чтения данных из таблицы в которой колонки разделены пробелами:
https://stackoverflow.com/questions/43058462/reading-a-variable-white-space-delimited-table-in-python

In [ ]:
path = 'e:/geomagnetic.txt'
data = np.genfromtxt(path)

Как объединить два массива с чередованием:
https://stackoverflow.com/questions/3678869/pythonic-way-to-combine-two-lists-in-an-alternating-fashion

In [ ]:
# Децимируем данные в 8 раз иначе очень долгое врем яобучения.
x = data[:,3][::decimation]
y = data[:,5][::decimation]

if normalize == True:
    m = max(np.amax(x), np.amax(y))
    x /= m
    y /= m

matplotlib.pyplot.plot(x)
matplotlib.pyplot.plot(y)

In [ ]:
# Объединение с чередованием наборов точек x и y.
z = np.zeros(len(x) + len(y))
z[0::2] = x
z[1::2] = y

#matplotlib.pyplot.xlim(right=100)
matplotlib.pyplot.plot(z)

## Подготовка данных для обучения

In [ ]:
def split(data, ratio):
    samples = int(len(data) * ratio)
    return data[:samples], data[samples:]

In [ ]:
def shift(data, dim):
    seq = []
    for i in range(len(data) - dim):
        seq.append(data[i:i+dim])
    return seq

In [ ]:
x_train, x_test = split(x, 0.8)
y_train, y_test = split(y, 0.8)
z_train, z_test = split(z, 0.8)

#x_train_in = y_train_in = shift(z_train, input_dim)
x_train_in = shift(x_train[0:], input_dim)
y_train_in = shift(y_train[0:], input_dim)

x_train_out = shift(x_train[1:], output_dim)
y_train_out = shift(y_train[1:], output_dim)

#assert len(x_input_train) == len(x_output_train)
#assert len(x_input_train) == len(y_output_train)

#print(len(x_input_train), len(x_output_train), len(y_input_train), len(y_output_train))

In [ ]:
# Проверка.
matplotlib.pyplot.xlim(right=80)
matplotlib.pyplot.plot(y_train)
matplotlib.pyplot.plot([y[1] for y in y_train_in])
matplotlib.pyplot.plot([y[1] for y in y_train_out])

## Обучение

In [ ]:
sess = tf.Session()
net_x = lstm_cell(sess, input_dim, hidden_dim, output_dim, learn_rate, optimizer)
net_y = lstm_cell(sess, input_dim, hidden_dim, output_dim, learn_rate, optimizer)

In [ ]:
begin = time.time()

for e in range(num_epochs):

    net_x.clear_state()
    net_y.clear_state()

    for x_in, x_out, y_in, y_out in zip(x_train_in, x_train_out, y_train_in, y_train_out):
        
        net_x.train([x_in], [x_out], num_repeat)
        net_y.train([y_in], [y_out], num_repeat)

    print(e, end=' ')

time_train = time.time() - begin
print(time_train)

net_x.save_all()
net_y.save_all()

## Предсказание

In [ ]:
#prev_seq = [x_output_train[-1]]
#prev_seq = np.zeros(input_dim)
#prev_seq[0::2] = x_output_train[-1]
#prev_seq[1::2] = y_output_train[-1]

x_prev = x_train_out[-1]
y_prev = y_train_out[-1]

x_pred = []
y_pred = []

net_x.restore_all()
net_y.restore_all()

for i in range(len(x_test)):

    x_out = net_x.test([x_prev])
    y_out = net_y.test([y_prev])

    x_prev = x_out[-1]
    y_prev = y_out[-1]

    #prev_seq[0::2] = x_out
    #prev_seq[1::2] = y_out

    x_pred.append(x_out[:,-1])
    y_pred.append(y_out[:,-1])
    
net_x.restore_all()
net_y.restore_all()

In [ ]:
fig, ax = matplotlib.pyplot.subplots(2, 1)

ax[0].plot(x_test)
ax[0].plot(x_pred, '--')

ax[1].plot(y_test)
ax[1].plot(y_pred, '--')

## SVD разложение

In [ ]:
net_x.restore_all()
n_x, _ = net_x.svd_compress(0.9, 0.9)

net_y.restore_all()
n_y, _ = net_y.svd_compress(0.9, 0.9)

print(n_x, n_y)

In [ ]:
net_x.restore_all()
net_y.restore_all()